# evaluating entities

In [ ]:
from evaluate import *
for exp in['events_pred']: #
    source_file=f'../dataset/plmarker/test_400window.jsonl'
    source_file=f'../dataset/plmarker/test_sent.jsonl'
    exp=f'{exp}'
    pred_ent_file=f'../dataset/plmarker/predictions/{exp}/ner_pred_results.json'
    pred_re_file=f'../dataset/plmarker/predictions/{exp}/re_pred_results.json'
    out_file=f'../dataset/plmarker/predictions/{exp}.json'
    plmarkerPred2json(source_file,pred_ent_file,pred_re_file,out_file)

In [ ]:
from evaluate import *
ann_path='../dataset/BRAT/test'


pred_file='../dataset/spert/test_spert_sent.json'
source_file='../dataset/spert/test_spert_sent.json'
out_dir='../dataset/spert/temp'

pred_file='../dataset/plmarker/predictions/events_pred.json'
source_file='../dataset/spert/test_spert_sent.json'
out_dir='../dataset/plmarker/predictions/BRAT'

# pred_file='../dataset/plmarker/predictions/400window_preEnt.json'
# source_file='../dataset/spert/test_spert_400window.json'
# out_dir='../dataset/plmarker/predictions/BRAT_REpreEnt'

offset_key,start_key,end_key="token_offsets",'start','end'
json2brat(source_file,ann_path,offset_key,start_key,end_key,pred_file,out_dir)

In [ ]:
from format_helper import *

for exp in ['llama2_events']: #'llama80b_events','t5_events','llama8b_events','gpt4_events'
    pred_file=f'../dataset/GenQA/predictions/{exp}.json'
    if 'llama' in exp or 'gpt' in exp:
        pred_file+='l'
    ann_path='../dataset/BRAT/test'
    out_dir=f'../dataset/GenQA/predictions/{exp}/pred'
    ref_dir=f'../dataset/GenQA/predictions/{exp}/ref'
    ref_file='../dataset/GenQA/test_events.json'
    if 'gpt' in exp:
        ref_file='../dataset/GenQA/gpt4_test_events.json'
    glmEvents2BRAT(pred_file,ann_path,out_dir,ref_dir,ref_file=ref_file)

In [ ]:
# scoring brat
from brat_scoring.scoring import score_brat
from format_helper import *

for exp in ['llama2_events']: #'llama80b_events',,'llama8b_events''gpt4_events'
    out_dir=f'../dataset/GenQA/predictions/{exp}/pred'
    ref_dir=f'../dataset/GenQA/predictions/{exp}/ref'
    score_dir=f'../dataset/GenQA/predictions/{exp}/'

exp='plmarker' #brat
out_dir=f'../dataset/{exp}/predictions/BRAT'
ref_dir=f'../dataset/BRAT/test'
score_dir=f'../dataset/{exp}/predictions/BRAT/score'


score_brat( 
    gold_dir = ref_dir,
    predict_dir = out_dir,
    labeled_args = [CHANGE,ASSERTION,SEVERITY],
    score_trig = "overlap",
    score_span = "overlap",
    score_labeled = "label",
    include_detailed = True,
    output_path =score_dir )

In [ ]:
# summarizing entity performance
from glob import glob
exps=['plmarker','spert','llama80b_events','llama8b_events','llama2_events','t5_events','gpt4_events']#,'spert']
for exp in exps:
        scores={
        "Drug,Trigger":[0,0,0],
        "Problem,Trigger":[0,0,0],
        "Problem,Assertion":[0,0,0],
        "Problem,Change":[0,0,0],
        "Problem,Severity":[0,0,0],
        "Problem,Anatomy":[0,0,0],
        "Problem,Characteristics":[0,0,0],
        "Problem,Duration":[0,0,0],
        "Problem,Frequency":[0,0,0],
        }
        for file in glob(f'../dataset/**/predictions/**/*.csv'):
            if exp in file and '_detailed' in file:
                print(file)
                lines=open(file).read().split('\n')[1:]
                out_ids=[]
                for line in lines:
                    if 'Problem,Trigger' in line and float(line.split(',')[-1])==0.5:
                        out_ids.append(line.split(',')[0])
                    
                for line in lines:
                    if line:
                        words=line.split(',')[1:]
                        key=','.join(words[:2])
                        if key not in scores:
                            #print(key)
                            continue
                        if any([id in line for id in out_ids]):
                            #print(line)
                            continue
                        if words[3]!='0':
                            for i in range(3):
                                scores[key][i]+=int(words[i+3])
        scores['Overall,-']=[0,0,0]
        for i in range(3):
            scores['Overall,-'][i]=sum([v[i] for k,v in scores.items()])

        with open(f'../performance/entity_{exp}.csv','w') as f:
            f.write('event,Argument,NT,NP,TP,P,R,F1\n')
            for key in scores:     
                f.write(key+','+','.join([str(s) for s in scores[key]]))
                P=scores[key][2]*100.0/scores[key][1] if scores[key][1] else 0
                R=scores[key][2]*100.0/scores[key][0] if scores[key][0] else 0
                F1=2*P*R/(P+R) if R+P else 0
                f.write(f",{round(P,1)},{round(R,1)},{round(F1,1)}\n")

# evaluating relations

In [ ]:
# converting from plmarker to spert json for evaluation
from evaluate import *
# for exp in['400window','sent']:
#     source_file=f'../dataset/plmarker/test_{exp}.jsonl'
#     exp=f'R15_re_{exp}'
#     pred_ent_file=f'/home/velvinfu/code/PL-Marker/experiments/{exp}/ner_pred_results.json'
#     pred_re_file=f'/home/velvinfu/code/PL-Marker/experiments/{exp}/re_pred_results.json'
#     out_file=f'../dataset/plmarker/predictions/{exp}.json'
#     plmarkerPred2json(source_file,pred_ent_file,pred_re_file,out_file)

# relation from predicted entities
source_file=f'../dataset/plmarker/predictions/pred_plmarker_400window.jsonl'
exp='400window_preEnt'
pred_ent_file=f'../dataset/plmarker/predictions/{exp}/ner_pred_results.json'
pred_re_file=f'../dataset/plmarker/predictions/{exp}/re_pred_results.json'
out_file=f'../dataset/plmarker/predictions/{exp}.json'
plmarkerPred2json(source_file,pred_ent_file,pred_re_file,out_file)

In [ ]:
# RE with QA format
from evaluate import *
spert_dir='../dataset/spert/test_spert_400window.json'#'../dataset/GenQA/predictions/t5_events_spert_400window.json'

llama_input_dir='../dataset/GenQA/predictions/t5_events_re.json'
llama_output_dir='../dataset/GenQA/predictions/t5_re_predEnt.json'
outfile='../dataset/GenQA/predictions/t5_re_predEnt_spert.json'

llama_input_dir='../dataset/GenQA/predictions/llama8b_events_re.json'
llama_output_dir='../dataset/GenQA/predictions/llama8b_re_PredEnt.jsonl'
outfile='../dataset/GenQA/predictions/llama8b_re_predEnt_spert.json'

QARe2json(spert_dir,llama_input_dir,llama_output_dir,outfile)

In [ ]:
# RE with the generative format
from evaluate import *
spert_dir='../dataset/spert/test_spert_400window.json'

llama_input_dir=''
llama_output_dir='../dataset/GenQA/predictions/t5_re_GenRe_pred.json'
outfile='../dataset/GenQA/predictions/t5_re_GenRe_pred_spert.json'

llama_input_dir=''
llama_output_dir='../dataset/GenQA/predictions/gpt4_re_GenRe_pred.json'
outfile='../dataset/GenQA/predictions/gpt4_re_GenRe_pred_spert.json'

# llama_input_dir='../dataset/GenQA/test_Genre.json'
# llama_output_dir='../dataset/GenQA/predictions/llama8b_re_GenRe_pred.jsonl'
# outfile='../dataset/GenQA/predictions/llama8b_re_GenRe_pred_spert.json'

# llama_input_dir=''
# llama_output_dir='../dataset/GenQA/predictions/t5_re_GenRe_predEnt_pred.json'
# outfile='../dataset/GenQA/predictions/t5_re_GenRe_predEnt_pred_spert.json'

# llama_input_dir='../dataset/GenQA/predictions/llama8b_events_spert_GenRe.json'
# llama_output_dir='../dataset/GenQA/predictions/llama8b_re_GenRe_predEnt_pred.jsonl'
# outfile='../dataset/GenQA/predictions/llama8b_re_GenRe_predEnt_pred_spert.json'


GenRe2json_predEnt(spert_dir,llama_input_dir,llama_output_dir,outfile)

In [ ]:
from evaluate import *
source_file='../dataset/spert/test_spert_400window.json'

models={
    # 'spert':['re_400window'],
    # 'plmarker':[
    #             'R15_re_400window',
    #             '400window_preEnt'
    #             ],
   'GenQA':[
    #    't5_re_predEnt_spert',
    #     'llama8b_re_predEnt_spert',
    #     't5_re_GenRe_pred_spert',
    #     'llama8b_re_GenRe_pred_spert',
    #    't5_re_GenRe_predEnt_pred_spert',
    #    'llama8b_re_GenRe_predEnt_pred_spert'
        'gpt4_re_GenRe_pred_spert'
   ]

}

for model,exps in models.items():
    for exp in exps:
        pred_file=f'../dataset/{model}/predictions/{exp}.json'
        out_file=f'../performance/{model}_{exp}.csv'
        error_file=f'../performance/{model}_{exp}_error.json'
        score_relations(pred_file,source_file,out_file,error_file)

# significance testing

In [2]:
import scipy.stats as stats
import numpy as np
from glob import glob
from tqdm import tqdm
def significance_test(ls1,ls2, num_iterations=10000):
    p_values=[]
    for i in tqdm(range(num_iterations)):
        t1 = np.random.choice(ls1, size=len(ls1), replace=True)
        t2 = np.random.choice(ls2, size=len(ls2), replace=True)
        p_value=stats.ttest_ind(t1,t2,alternative='less').pvalue
        p_values.append(p_value)
    p_value=np.mean(p_values)
    # if p_value<=0.05:
    #     return p_value
    return p_value

filenames=glob('../dataset/BRAT/test/*.txt')
filenames=[f.split('/')[-1].split('.')[0] for f in filenames]

exps=['spert','plmarker','t5_events','llama2_events','llama8b_events','gpt4_events']

keys=[
    # '','Drug,Trigger','Problem,Trigger','Problem,Assertion','Problem,Change','Problem,Severity',
    # 'Problem,Anatomy','Problem,Characteristics','Problem,Duration',
    'Problem,Frequency'
]

def get_scores(key,file):
    scores={}
    for f in filenames:
        scores[f]=[0,0,0]
    for l in open(file).readlines()[1:]:
        if l.strip() and ',' in l and key in l:
            doc_id=l.split(',')[0]
            for i in range(3):
                scores[doc_id][i]+=int(l.split(',')[i+4])
    
    results=[]
    for k,(NT,NP,TP) in scores.items():
        P=TP/NP if NP else 0
        R=TP/NT if NT else 0
        F1=(P*R)*2/(P+R) if (P*R) else 0
        results.append(F1)
    return results

candidate_files=glob(f'../dataset/**/predictions/**/*.csv')
with open('../performance/ent_significance.csv','w') as f:
    for key in keys:
        f.write(f'\n\n{key}\n,')
        f.write(','.join([key for key in exps]))
        for model1 in exps:
            file1=[f for f in candidate_files if model1 in f and 'detailed' in f]
            assert len(file1)==1
            file1=file1[0]
            f.write(f'\n{model1},')
            scores1=get_scores(key,file1)
            for model2 in exps:
                file2=[f for f in candidate_files if model2 in f and 'detailed' in f]
                assert len(file2)==1,(model2,file2)
                file2=file2[0]
                scores2=get_scores(key,file2)
                f.write(f'{significance_test(scores1,scores2)},')

 10%|▉         | 966/10000 [00:00<00:06, 1291.53it/s]

100%|██████████| 10000/10000 [00:05<00:00, 1839.98it/s]


In [12]:
from config import *
import scipy.stats as stats
import numpy as np

def significance_test(ls1,ls2, num_iterations=10000):
    p_values=[]
    for i in range(num_iterations):
        t1 = np.random.choice(ls1, size=len(ls1), replace=True)
        t2 = np.random.choice(ls2, size=len(ls2), replace=True)
        p_value=stats.ttest_ind(t1,t2,alternative='less').pvalue
        p_values.append(p_value)
    p_value=np.mean(p_values)
    return p_value

experiments={
    'intra_gold':['plmarker_R15_re_400window_intra','GenQA_llama8b_re_GenRe_pred_spert_intra','llama8b_re_intra','t5_re_intra','GenQA_gpt4_re_GenRe_pred_spert_intra'],
    # 'intra_pred':['spert_re_400window_intra','plmarker_400window_preEnt_intra',
    #               'GenQA_llama8b_re_GenRe_predEnt_pred_spert_intra','GenQA_t5_re_GenRe_predEnt_pred_spert_intra',
    #               'GenQA_t5_re_predEnt_spert_intra','GenQA_llama8b_re_predEnt_spert_intra'],
    'all_gold':['plmarker_R15_re_400window_any','GenQA_llama8b_re_GenRe_pred_spert_any','t5_re_any','GenQA_gpt4_re_GenRe_pred_spert_intra'],
    # 'all_pred':['spert_re_400window_intra','plmarker_400window_preEnt_any',
    #             'GenQA_llama8b_re_GenRe_predEnt_pred_spert_any','GenQA_t5_re_GenRe_predEnt_pred_spert_any',
    #   'GenQA_t5_re_predEnt_spert_any','GenQA_llama8b_re_predEnt_spert_any']
}
keys=['overall','AdminFor','NotAdminBecause','Causes','Improves','Worsens','PIP']

def get_scores(key,file):
    scores=[float(l.split(',')[-1]) for l in open(file).readlines()[1:] if l.strip() and ',' in l and (key in l or relation_names_reversed.get(key,'Overall') in l)]
    return scores

for title, exps in experiments.items():
    with open(f'../performance/rel_significance_{title}.csv','w') as f:
        for key in keys:
            f.write(f'\n\n{key}\n,')
            f.write(','.join([key for key in exps]))
            for model1 in exps:
                file1=f'../performance/{model1}_by_docs.csv'
                f.write(f'\n{model1},')
                scores1=get_scores(key,file1)
                for model2 in exps:
                    file2=f'../performance/{model2}_by_docs.csv'
                    scores2=get_scores(key,file2)
                    p_value=significance_test(scores1,scores2)
                    f.write(f'{p_value},')
                    #print(p_value)

# plot performance vs distance

In [ ]:
import scipy.stats as stats
import numpy as np
def get_trigger_scores(file):
    scores={}
    for f in filenames:
        scores[f]=[0,0,0]
    for l in open(file).readlines()[1:]:
        if l.strip() and ',' in l and 'Trigger' in l:
            doc_id=l.split(',')[0]
            for i in range(3):
                scores[doc_id][i]+=int(l.split(',')[i+4])
    
    results={}
    for k,(NT,NP,TP) in scores.items():
        P=TP/NP if NP else 0
        R=TP/NT if NT else 0
        F1=(P*R)*2/(P+R) if (P*R) else 0
        results[k]=F1
    return results

# average number of triggers per 100 tokens
filename='../dataset/spert/test_spert_note.json'
source=json.loads(open(filename).read())
count={}
for dic in source:
    triggers=len([ent for ent in dic['entities'] if 'Drug' in ent['type'] or 'Problem' in ent['type']])
    count[dic['id']]=int(triggers*100.0/len(dic['tokens']))

exps=['spert','plmarker','t5_events','llama8b_events','llama80b_events']

candidate_files=glob(f'../dataset/**/predictions/**/*.csv')
with open('../performance/ent_by_dis.csv','w') as f:
    for i in range(4,9):
        f.write(f',{i}')
    for model in exps:
        file=[f for f in candidate_files if model in f and 'detailed' in f]
        assert len(file)==1
        file=file[0]
        f.write(f'\n{model}')  
        scores=get_trigger_scores(file)
        for i in range(4,9):
            score=np.mean([scores[f] for f,c in count.items() if c==i])
            f.write(f',{score}')